In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from gensim.models import KeyedVectors

In [3]:
ft=KeyedVectors.load("/content/gdrive/MyDrive/FYP/CAA_NRC/FastText/ft")

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])


In [5]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [6]:
import tensorflow as tf
maxlen=maxi
max_words=len(ft.wv.vocab.keys())
print(max_words)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

2123


In [7]:
len(ft.wv.vocab.keys())

2123

In [8]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  
  if word in ft and i < max_words:
    embed_vector=ft[word]
    
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,LSTM,Dense,GRU
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(GRU(128,return_sequences=True))
model.add(GRU(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))

In [ ]:
import time
start = time.time()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/5
37/37 [==============================] - 21s 459ms/step - loss: 0.4716 - accuracy: 0.5940
Epoch 2/5
37/37 [==============================] - 10s 279ms/step - loss: 0.4468 - accuracy: 0.6353
Epoch 3/5
37/37 [==============================] - 10s 280ms/step - loss: 0.4105 - accuracy: 0.6740
Epoch 4/5
37/37 [==============================] - 10s 278ms/step - loss: 0.3973 - accuracy: 0.6976
Epoch 5/5
37/37 [==============================] - 10s 281ms/step - loss: 0.3724 - accuracy: 0.7428
Training time: 62.30335307121277s


In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))


In [ ]:
import numpy as np
y_pred=model.predict(X_test)
# y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.6675257731958762
0.6658003719633622
0.6822531452535264
0.6766039517522762
